In [2]:
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken
import dotenv

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [4]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
query = "How do I execute a command in a running docker container?"

In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

In [13]:
print(response['hits']['hits'][0]['_score'])

83.03066


In [14]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index=index_name, body=search_query)

In [15]:
print(response['hits']['hits'][2]['_source']['question'])

How do I copy files from a different folder into docker container’s working directory?


In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [17]:
context = ""
for doc in response['hits']['hits']:
    context = context + f'{context_template.format(question=doc["_source"]["question"], text=doc["_source"]["text"])}\n\n'

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [19]:
prompt = prompt_template.format(question=query, context=context).strip()

In [20]:
len(prompt)

1462

In [21]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
len(encoding.encode(prompt))

322

In [23]:
import ollama

In [24]:
def llm(prompt):
    response = ollama.chat(model='gemma2:9b', 
                           messages=[{
                               'role': 'user',
                               'content': prompt,
                            },])    
    return response['message']['content']

In [25]:
response = llm(prompt)

In [26]:
print(response)

If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash 



